In [12]:
import pandas as pd
import polars as pl

In [44]:
cn = pd.read_csv("cn_data.csv")
pn = pd.read_csv("pn_data.csv")

In [45]:
prod = pl.read_csv(r"C:\WorkSpace\Python\DA_basics\Data\products.csv", infer_schema_length=10000,truncate_ragged_lines=True)

In [46]:
cn = cn.sort_values(["key","vendor_cn_date"]).reset_index(drop=True)
pn = pn.sort_values(["key","date"]).reset_index(drop=True)

In [47]:
len(cn["key"].unique())

4761

In [48]:
len(pn["key"].unique())

4673

In [49]:
uniq_key = set()
uniq_key.update(cn["key"].unique())
uniq_key.update(pn["key"].unique())
uniq_key = sorted(uniq_key)

In [50]:
len(uniq_key)

6572

In [52]:
columns = ["vendor_id", "ws_code", "batch_number", "product_name", "cn_number", "cn_date", "cn_ref_date", "cn_qty",
           "pn_number","pn_date", "pn_qty", "finalqty", "Note", "key"]
temp_df = pd.DataFrame(columns=columns)
Note = ["CN pending", "Excess CN received", "Exact Match"]

i, j = 0, 0

for key in uniq_key:
  count = 0
  vendor_id, ws_code, batch_num = key.split("_")
  while(count != 1):
    
    if i < len(pn["key"]) and j < len(cn["key"]) and pn["key"][i] == key and cn["key"][j] == key:
        k = pn["quantity"][i] - cn["billed_quantity"][j]
        if k > 0:
          temp_df.loc[len(temp_df)] = [vendor_id,ws_code,batch_num,prod.filter(pl.col("ws_code") == int(ws_code))[0,"product_name"], cn["debit_note_number"][j],cn["vendor_cn_date"][j],"",cn["billed_quantity"][j],pn["pick_note_number"][i],pn["date"][i], pn["quantity"][i],k, Note[0], key]
          pn.loc[i, "quantity"] = k

          j += 1
        elif k < 0:
          temp_df.loc[len(temp_df)] = [vendor_id,ws_code,batch_num,prod.filter(pl.col("ws_code") == int(ws_code))[0,"product_name"], cn["debit_note_number"][j],cn["vendor_cn_date"][j],"",cn["billed_quantity"][j],pn["pick_note_number"][i],pn["date"][i], pn["quantity"][i],k * -1, Note[1], key]
          cn.loc[j, "billed_quantity"] = k*-1

          i += 1
        else:
          temp_df.loc[len(temp_df)] = [vendor_id,ws_code,batch_num,prod.filter(pl.col("ws_code") == int(ws_code))[0,"product_name"], cn["debit_note_number"][j],cn["vendor_cn_date"][j],"",cn["billed_quantity"][j],pn["pick_note_number"][i],pn["date"][i], pn["quantity"][i],k, Note[2], key]
          i += 1
          j += 1

    elif i < len(pn["key"]) and pn["key"][i] == key:
        # CN -> " "" 
        temp_df.loc[len(temp_df)] = [vendor_id,ws_code,batch_num,prod.filter(pl.col("ws_code") == int(ws_code))[0,"product_name"], " "," ",""," ",pn["pick_note_number"][i],pn["date"][i], pn["quantity"][i],k, Note[0], key]
          
        i += 1
    elif j < len(cn["key"]) and cn["key"][j] == key:
        
        temp_df.loc[len(temp_df)] = [vendor_id,ws_code,batch_num,prod.filter(pl.col("ws_code") == int(ws_code))[0,"product_name"], cn["debit_note_number"][j],cn["vendor_cn_date"][j],"",cn["billed_quantity"][j],""," ", " ",k, Note[1], key]
          
        j += 1
    else:
        count = 1


In [53]:
temp_df

,vendor_id,ws_code,batch_number,product_name,cn_number,cn_date,cn_ref_date,cn_qty,pn_number,pn_date,pn_qty,finalqty,Note,key
0,322,11299,SIF1190A,REPACE 25MG TAB 1X10,DN201006195,2024-11-25,,1,PR201008895,23-11-2024,1,0,Exact Match,322_11299_SIF1190A
1,322,11303,SIF1510A,REPACE H TAB 1X10,DN201006163,2024-11-22,,1,PR201008882,20-11-2024,1,0,Exact Match,322_11303_SIF1510A
2,322,1178,FHA0249,AZTOGOLD 10MG TAB 1X10,DN201006186,2024-11-19,,6,PR201008755,16-11-2024,6,0,Exact Match,322_1178_FHA0249
3,322,13214,I304954,TENGLYN M 1000MG TAB 1X15,DN301006894,2025-01-17,,2,,,,0,Excess CN received,322_13214_I304954
4,322,13460,SIF1920A,TIGATEL 20MG TAB 1X10,DN201006646,2024-12-28,,6,PR201009347,28-12-2024,6,0,Exact Match,322_13460_SIF1920A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8934,432,21898,B3020,BSC CHARCOAL FACEWASH 100GM,DN201006386,2024-12-04,,152,PR201008968,04-12-2024,152,0,Exact Match,432_21898_B3020
8935,432,21902,BSHFD24148,BSC CHARCOAL FOAM 425GM,DN201006386,2024-12-04,,50,PR201008968,04-12-2024,50,0,Exact Match,432_21902_BSHFD24148
8936,432,21902,BSHFD24149,BSC CHARCOAL FOAM 425GM,DN201006386,2024-12-04,,8,PR201008968,04-12-2024,8,0,Exact Match,432_21902_BSHFD24149
8937,437,23009,VEX0248S,7VIK CHATPATA SYP 100ML,DN201006136,2024-11-19,,1,PR201008819,19-11-2024,1,0,Exact Match,437_23009_VEX0248S


In [ ]:
len(temp_df["key"].unique())

In [114]:
temp_df.to_csv("output.csv")